In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/brain-tumor-mri-dataset


In [12]:
import os
import cv2

In [44]:
os.listdir(path+'/Training')

['pituitary', 'notumor', 'meningioma', 'glioma']

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # fill_mode='nearest'
)

# # Assuming 'img' is a loaded image in numpy array format
# aug_iter = datagen.flow(img.reshape((1,)+img.shape))
# augmented_images = [next(aug_iter)[0].astype('uint8') for i in range(10)]


In [66]:
imagesdone = 1
count = 1

In [67]:
for dir in os.listdir(path + "/Training"):
  print(dir)
  for img in os.listdir(path + "/Training/"+dir):
    print(img)
    im = cv2.imread(path + "/Training/"+dir+"/"+img)
    # print(im)
    # img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    aug_iter = datagen.flow(im.reshape((1,)+im.shape))
    augmented_images = [next(aug_iter)[0].astype('uint8') for i in range(7)]
    for i in range(len(augmented_images)):
      cv2.imwrite(f'augmented_dataset/{dir}/augmented_image_{img}_{i}.jpg', augmented_images[i])
    cv2.imwrite(f'augmented_dataset/{dir}/original_image_{img}.jpg', im)
    print("---"+str(imagesdone))
    imagesdone+=1
    print(len(os.listdir(f'augmented_dataset/{dir}')))


Streaming output truncated to the last 5000 lines.
7952
Tr-me_1236.jpg
---4047
7960
Tr-me_0255.jpg
---4048
7968
Tr-me_0526.jpg
---4049
7976
Tr-me_1263.jpg
---4050
7984
Tr-me_0673.jpg
---4051
7992
Tr-me_0383.jpg
---4052
8000
Tr-me_0597.jpg
---4053
8008
Tr-me_0242.jpg
---4054
8016
Tr-me_1257.jpg
---4055
8024
Tr-me_0944.jpg
---4056
8032
Tr-me_0730.jpg
---4057
8040
Tr-me_0855.jpg
---4058
8048
Tr-me_0631.jpg
---4059
8056
Tr-me_1316.jpg
---4060
8064
Tr-me_0674.jpg
---4061
8072
Tr-me_0117.jpg
---4062
8080
Tr-me_0954.jpg
---4063
8088
Tr-me_0069.jpg
---4064
8096
Tr-me_0802.jpg
---4065
8104
Tr-me_0719.jpg
---4066
8112
Tr-me_0960.jpg
---4067
8120
Tr-me_1028.jpg
---4068
8128
Tr-me_0475.jpg
---4069
8136
Tr-me_0737.jpg
---4070
8144
Tr-me_0824.jpg
---4071
8152
Tr-me_0083.jpg
---4072
8160
Tr-me_0453.jpg
---4073
8168
Tr-me_0894.jpg
---4074
8176
Tr-me_0847.jpg
---4075
8184
Tr-me_0010.jpg
---4076
8192
Tr-meTr_0002.jpg
---4077
8200
Tr-me_1077.jpg
---4078
8208
Tr-me_0199.jpg
---4079
8216
Tr-me_0278.jpg
---

In [68]:
%pip install split-folders

In [77]:
import splitfolders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.import splitfolders

# Define input and output directories
input_folder = f"augmented_dataset"  # Folder containing subdirectories of images
# output_folder = "dataset"  # Folder where train/test/val will be created

# Split into train (80%), val (10%), test (10%)
# splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0, 0.5, 0.5), group_prefix=None)

# input_folder = f"{path}/Testing"  # Folder containing subdirectories of images
output_folder = "augds_"  # Folder where train/test/val will be created

# Split into train (80%), val (10%), test (10%)
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.1, 0.1), group_prefix=None)

Copying files: 45696 files [00:39, 1166.93 files/s]


In [80]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Directory paths (update these paths based on your local setup)
train_dir = f'./dataset/train'
val_dir = './dataset/val'
test_dir = './dataset/test'

# Data augmentation for training; simply rescale for validation/test
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    zoom_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained DenseNet121 model without the top classification layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Helps prevent overfitting
predictions = Dense(5, activation='softmax')(x)  # categorical classification: pneumonia or normal

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model for initial training
for layer in base_model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
# model.summary()

# Train the model
# history = model.fit(
#     train_generator,
#     epochs=EPOCHS,
#     validation_data=val_generator
# )

# Optionally, unfreeze some layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Fine-tune with model.fit(...)


Found 36555 images belonging to 5 classes.
Found 5879 images belonging to 5 classes.
Found 4573 images belonging to 5 classes.


In [ ]:
for i in range(30):
  model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator
  )
  # model.save(f'braintumor_densenet121_aug_{i}.keras')
  model.save(f'/content/drive/MyDrive/braintumor_densenet121__aug_{i}.keras')

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_1290']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1143/1143 ━━━━━━━━━━━━━━━━━━━━ 927s 653ms/step - accuracy: 0.7065 - loss: 0.7989 - val_accuracy: 0.9507 - val_loss: 0.1418
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 598s 523ms/step - accuracy: 0.9469 - loss: 0.1546 - val_accuracy: 0.9794 - val_loss: 0.0624
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 591s 517ms/step - accuracy: 0.9723 - loss: 0.0810 - val_accuracy: 0.9852 - val_loss: 0.0434
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 586s 513ms/step - accuracy: 0.9861 - loss: 0.0430 - val_accuracy: 0.9903 - val_loss: 0.0281
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 598s 523ms/step - accuracy: 0.9902 - loss: 0.0281 - val_accuracy: 0.9935 - val_loss: 0.0199
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 602s 526ms/step - accuracy: 0.9946 - loss: 0.0191 - val_accuracy: 0.9951 - val_loss: 0.0170
1143/1143 ━━━━━━━━━━━━━━━━━━━━ 606s 530ms/step - accuracy: 0.9961 - loss: 0.0130 - val_accuracy: 0.9973 - val_loss: 0.0088
 893/1143 ━━━━━━━━━━━━━━━━━━━━ 2:08 516ms/step - accuracy: 0.9959 - loss: 0.0126